In [1]:
import psycopg2
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

In [2]:
# Загрузить переменные окружения

load_dotenv()

# Получить переменную окружения
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

In [3]:
connection = psycopg2.connect(
    database=db_name, user=db_user, password=db_password, host=db_host, port=db_port
)
cursor = connection.cursor()


In [4]:
df = pd.read_sql(
    """SELECT * FROM "user" LIMIT 10 """,
    con=f"postgresql://{db_user}:{db_password}@"
        f"{db_host}:{db_port}/{db_name}"
)

df

,id,gender,age,country,city,exp_group,os,source
0,1,Male,38,Guernsey,South Jeffreymouth,3,iOS,gonzalez.com
1,2,Female,33,Saint Lucia,Ashleybury,1,iOS,powell.biz
2,3,Male,30,Vanuatu,Davidton,3,Windows,lam.net
3,4,Female,51,Singapore,South Ashley,1,Windows,lozano-anderson.com
4,5,Male,45,Slovenia,Alexisberg,0,Windows,marsh.biz
5,6,Male,45,Japan,Stanleyview,0,Android,hodges.com
6,7,Male,30,Iceland,East Michaelville,1,Android,lewis.com
7,8,Female,41,Uganda,Davisberg,1,Windows,rodriguez-campos.org
8,9,Female,45,Maldives,East Jonathanside,3,iOS,robinson-murphy.com
9,10,Male,43,Iraq,South Alexberg,2,iOS,fernandez.com


In [5]:
df = pd.read_sql('''
SELECT * FROM post;
''',connection)
df

/tmp/ipykernel_42068/996451875.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,id,text,topic
0,1,Sort people key. Professor laugh worry artist....,leg
1,2,Class game stay do machine institution.\nWhile...,officer
2,3,Alone choose program increase world society.\n...,foot
3,4,Yeah price health trade could strong. Clearly ...,work
4,5,Why world late. Stay ball eight nice beyond no...,generation
...,...,...,...
245,246,Mother avoid health husband Republican interes...,individual
246,247,Computer federal answer particular decision do...,son
247,248,However agree inside manage. Teach within occu...,worry
248,249,Boy next deep indicate simply politics buildin...,key


In [6]:
t = df['topic'].values
l = ""
for i in t:
    if len(i)>len(l):
        l = i
print(len(t),l)

250 administration


In [7]:
df = pd.read_sql('''SELECT * FROM "user" AS u
                    WHERE u.age > 30 AND u.os = 'iOS';
                 ''',connection)
print(df['id'].values[0], df['city'].values[0])

1 South Jeffreymouth


/tmp/ipykernel_42068/1600272196.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''SELECT * FROM "user" AS u


In [8]:
df = pd.read_sql('''
SELECT * FROM "user" as u
WHERE u.exp_group <> 3 AND u.exp_group <> 0 AND u.city = 'South Ashley' AND COUNTRY !='Russia';
''',
connection)
df

/tmp/ipykernel_42068/103286596.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,id,gender,age,country,city,exp_group,os,source
0,4,Female,51,Singapore,South Ashley,1,Windows,lozano-anderson.com


In [9]:
df = pd.read_sql('''
SELECT age, country FROM "user" as u
WHERE country = 'Japan';
''',
connection)
np.around(df['age'].mean(),2)

/tmp/ipykernel_42068/3885211706.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


np.float64(49.33)

In [10]:
df = pd.read_sql('''
SELECT exp_group, os, COUNT(*), MIN(age), MAX(age) FROM "user" 
GROUP BY exp_group, os
ORDER BY "user".exp_group;
                 
''',connection)
df = df.to_csv('./csv_group_by_1.csv',sep=',', header=['exp_group','os','total_users','min_age','max_age'])
pd.read_csv('./csv_group_by_1.csv')

/tmp/ipykernel_42068/3088059918.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,Unnamed: 0,exp_group,os,total_users,min_age,max_age
0,0,0,Android,38,23,60
1,1,0,iOS,36,21,59
2,2,0,Windows,43,18,60
3,3,1,iOS,43,18,60
4,4,1,Windows,41,19,58
5,5,1,Android,41,18,60
6,6,2,Android,46,18,60
7,7,2,Windows,34,18,59
8,8,2,iOS,47,18,60
9,9,3,iOS,43,21,59


In [11]:
df = pd.read_sql('''
SELECT topic, MAX(LENGTH(text)) FROM post
GROUP BY topic
''',connection)
df

/tmp/ipykernel_42068/2001499091.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,topic,max
0,age,72
1,again,160
2,effort,163
3,concern,185
4,son,148
...,...,...
214,after,107
215,prevent,194
216,activity,187
217,news,171


In [12]:
df = pd.read_sql('''SELECT country, COUNT(*) as c FROM "user" AS u
                    GROUP BY country 
                    HAVING COUNT(*) > 3 
''',connection)
df

/tmp/ipykernel_42068/3538815549.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''SELECT country, COUNT(*) as c FROM "user" AS u


,country,c
0,Indonesia,4
1,Kiribati,4
2,Cambodia,5
3,Papua New Guinea,4
4,Singapore,4
5,San Marino,6
6,Cayman Islands,4
7,Tokelau,5
8,Cape Verde,4
9,Iceland,4


In [13]:
df = pd.read_sql('''SELECT country, exp_group, COUNT(id)
FROM "user"
GROUP BY country, exp_group;
''',connection)
df

/tmp/ipykernel_42068/1330726539.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''SELECT country, exp_group, COUNT(id)


,country,exp_group,count
0,Indonesia,3,1
1,Puerto Rico,2,2
2,India,0,1
3,Benin,3,1
4,Mozambique,1,1
...,...,...,...
378,French Polynesia,3,1
379,Eritrea,3,1
380,Tuvalu,0,1
381,Slovenia,3,1


In [14]:
df = pd.read_sql('''
SELECT * FROM "user" 
WHERE city = 'Davisberg'
''', connection)
df

/tmp/ipykernel_42068/1605592633.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,id,gender,age,country,city,exp_group,os,source
0,8,Female,41,Uganda,Davisberg,1,Windows,rodriguez-campos.org


In [15]:
df = pd.read_sql('''SELECT topic, COUNT(*) FROM post
                    GROUP BY topic
                    ORDER BY COUNT(*) DESC
                    LIMIT 3;  
                ''',connection)
df

/tmp/ipykernel_42068/2080583341.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''SELECT topic, COUNT(*) FROM post


,topic,count
0,fund,3
1,cover,3
2,value,3


In [16]:
df = pd.read_sql('''SELECT id,age, exp_group,COUNT(*)  FROM "user"
                    where country = 'Poland'
                    GROUP BY id,age, exp_group
                    ORDER BY age DESC, exp_group ASC
                ''',connection)
df['id'][2]

/tmp/ipykernel_42068/158732818.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''SELECT id,age, exp_group,COUNT(*)  FROM "user"


np.int64(361)

In [17]:
df = pd.read_sql('''
SELECT u.id , f.user_id, p.id, f.post_id , f.time , u.os, u.age FROM "user" u 
JOIN feed_action f ON u.id = f.user_id
JOIN post p on p.id = f.user_id
ORDER BY f.time
''',connection)
df

/tmp/ipykernel_42068/3882976520.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,id,user_id,id,post_id,time,os,age
0,4,4,4,47,2024-01-01 01:25:34.678305,Windows,51
1,8,8,8,45,2024-01-01 02:34:56.177620,Windows,41
2,33,33,33,17,2024-01-01 05:51:09.023381,Windows,27
3,64,64,64,24,2024-01-01 09:37:56.759130,iOS,42
4,27,27,27,41,2024-01-01 14:00:21.432849,iOS,42
...,...,...,...,...,...,...,...
995,32,32,32,14,2024-10-10 02:54:51.023879,Windows,53
996,31,31,31,17,2024-10-10 18:25:35.963682,Windows,46
997,43,43,43,13,2024-10-11 02:07:05.020539,Android,49
998,21,21,21,4,2024-10-11 10:01:36.602371,Android,36


In [18]:
df = pd.read_sql('''
SELECT f.post_id, COUNT(f.action), MAX(f.time) AS c FROM post p
JOIN feed_action f on p.id = f.post_id
GROUP BY f.post_id
ORDER BY c DESC;
''',connection)
df

/tmp/ipykernel_42068/2543332372.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


,post_id,count,c
0,34,20,2024-10-11 14:44:47.003165
1,4,22,2024-10-11 10:01:36.602371
2,13,23,2024-10-11 02:07:05.020539
3,17,24,2024-10-10 18:25:35.963682
4,14,29,2024-10-10 02:54:51.023879
5,49,15,2024-10-10 00:54:24.632685
6,27,23,2024-10-09 17:14:30.131902
7,39,18,2024-10-09 16:16:48.365351
8,42,17,2024-10-09 13:27:49.645449
9,48,15,2024-10-09 10:30:07.695489
